# Imports and loading data

In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
df = pd.read_csv("../proofs.csv")
df.head()

,url,body
0,https://proofwiki.org/wiki/Left_and_Right_Inve...,"== Theorem ==\n\nLet $\struct {S, \circ}$ be a..."
1,https://proofwiki.org/wiki/Lower_Bound_is_Lowe...,{{mergeto|Lower Bound for Subset}}\n== Theorem...
2,https://proofwiki.org/wiki/Mapping/Examples/Ma...,== Example of [[Definition:Mapping|Mapping]] =...
3,https://proofwiki.org/wiki/Closed_Form_for_Hex...,== Theorem ==\n\nThe [[Definition:Closed-Form ...
4,https://proofwiki.org/wiki/770,{{NumberPageLink|prev = 769|next = 771}}\n\n==...


In [3]:
bad_urls = [
    "https://proofwiki.org/wiki/Main_Page"
]
df = df[~df.url.isin(bad_urls)] # filter out bad urls

# Main cleaning script

In [4]:
whitespace = re.compile(' +')
text_latex_tag = re.compile('\\text *\{.*?\}')
inline_equation = re.compile("\$(.|\n)*?\$")
begin_eqn = re.compile("\{\{begin-eqn.*?\}\}(.|\n)*?\{\{end-eqn.*?\}\}")
begin_axiom = re.compile("\{\{begin-axiom.*?\}\}(.|\n)*?\{\{end-axiom.*?\}\}")
begin_tableau = re.compile("\{\{BeginTableau.*?\}\}(.|\n)*?\{\{EndTableau.*?\}\}")
bad_tags = re.compile("|".join([
    "<onlyinclude>",
    "</onlyinclude>",
    "<tt>",
    "</tt>",
    "<pre>(.|\n)*?</pre>",
    "<code>(.|\n)*?</code>",
    "<br>",
    "<br/>",
    "</br>",
    "<br />",
    "<nowiki>",
    "</nowiki>",
    "<!--(.|\n)*?-->",
    "<section.*?>",
    "</section>",
    "<ref>",
    "</ref>",
    "<span.*?>",
    "</span>",
    "<h1.*?>",
    "</h1>",
    "<includeonly>",
    "</includeonly>",
    "<references/>",
    "\{\{Delete\}\}",
    "\{\{Namedfor.*?\}\}",
    "\{\{NamedforDef.*?\}\}",
    "\{\{Expand\}\}",
    "\{\{qed.*?\}\}",
    "\{\{handwaving\}\}",
    "\{\{explain\}\}",
    "\{\{BookReference.*\}\}",
    "\{\{Qed\}\}",
    "\{\{ProofWanted\}\}",
    "\{\{MissingLinks\}\}",
    "\{\{Proofread\}\}",
    "\{\{proofread\}\}",
    "\{\{proof wanted\}\}",
    "\{\{finish\}\}",
    "\{\{refactor\}\}",
    "\{\{tidy\}\}",
    "\{\{EuclidPropLink.*?\}\}",
    "\[\[Category:.*?\]\]",
]))
bold_italics = re.compile("\'{5}(.*?)\'{5}|")
bold = re.compile("\'{3}(.*?)\'{3}")
italics = re.compile("\'{2}(.*?)\'{2}")
link_text = re.compile("\[\[.*?\|(.*?)\]\]")
other_links = re.compile("\[\[.*?\]\]")
link_to_category = re.compile("\{\{LinkToCategory\|.*?\|(.*?)\}\}")
def_of = re.compile("\{\{Defof\|(.*?)\}\}")
redirect = re.compile("^#REDIRECT")
sections = re.compile("={6} *(.*?) *={6}|={5} *(.*?) *={5}|={4} *(.*?) *={4}|={3} *(.*?) *={3}|={2} *(.*?) *={2}")
bulleted_list = re.compile("^\*{1,}")
numbered_list = re.compile("^#{1,}")
indented = re.compile("^:{1,}")
templates = [
    (re.compile("\{\{AimForCont\}\}"), "aiming for a contradiction, suppose"),
    (re.compile("\{\{iff\}\}"), "if and only if"),
    (re.compile("\{\{WLOG\}\}"), "without loss of generality"),
    (re.compile("\{\{WRT.*?\}\}"), "with respect to"),
    (re.compile("\{\{TFAE\}\}"), "the following are equivalent"),
    (re.compile("\{\{RHS\}\}"), "right-hand side"),
    (re.compile("\{\{LHS\}\}"), "left-hand side"),
]
other_templates = re.compile("\{\{.*?\}\}")

def clean(text):
    text = re.sub(whitespace, " ", text) # remove unnecessary whitespace
    
    text = re.sub(text_latex_tag, "\text{}", text) # weird case where $\text{$x = 4$}$ parses
    
    text = re.sub(inline_equation, "{inline-equation}", text) # equations
    text = re.sub(begin_eqn, "{equation}", text) # equations
    text = re.sub(begin_axiom, "{equation}", text) # axioms
    text = re.sub(begin_tableau, "", text) # random proofs called "tableau proofs"
    
    text = re.sub(bad_tags, "", text) # bad tags
    
    text = re.sub(bold_italics, r"\1", text) # bold & italics
    text = re.sub(bold, r"\1", text) # bold
    text = re.sub(italics, r"\1", text) # italics
    
    text = re.sub(link_text, r"\1", text) # link with text
    text = re.sub(other_links, "{theorem}", text) # other links
    
    text = re.sub(link_to_category, r"\1", text) # {{LinkToCategory|...}} template
    text = re.sub(def_of, r"\1", text) # {{Defof|...}} template
    
    for template, actual in templates:
        text = re.sub(template, actual, text)
    text = re.sub(other_templates, "", text) # other templates
    
    sentences = text.split('\n') # split by lines
    for i, sentence in enumerate(sentences):
        sentence = re.sub(redirect, "", sentence) # redirect

        # TODO: there has to be a better regex way to do this part
        sentence = re.sub(sections, r"== \1\2\3\4\5 ==", sentence) # uniform section headings
        
        sentence = re.sub(bulleted_list, "", sentence) # bulleted list
        sentence = re.sub(numbered_list, "", sentence) # numbered list
        sentence = re.sub(indented, "", sentence) # indented
        sentence = sentence.strip()
        sentences[i] = sentence
    sentences = [sentence for sentence in sentences if len(sentence) > 0]
    text = "\n".join(sentences)
    
    return text

text = """
== Proof 1 ==
unnecessary     whitespace
'''''both'''''
'''bold''' 
''italics''
$inline equation$
$
multiline inline equation
$
{{begin-eqn}}begin-eqn{{end-eqn}}
{{begin-eqn}}
multiline begin-eqn
{{end-eqn}}
[[url|link text]]
hello
*** bulleted list
***bulleted list
#### numbered list
##numbered list
:::indent
::: indent
#REDIRECT
=Section 1=
==Section 2==
===Section 3===
====Section 4====
=====Section 5=====
======Section 6======
"""
clean(text)

'== Proof 1 ==\nunnecessary whitespace\nboth\nbold\nitalics\n{inline-equation}\n{inline-equation}\n{equation}\n{equation}\nlink text\nhello\nbulleted list\nbulleted list\nnumbered list\nnumbered list\nindent\nindent\n=Section 1=\n== Section 2 ==\n== Section 3 ==\n== Section 4 ==\n== Section 5 ==\n== Section 6 =='

In [5]:
df["body_cleaned"] = df.body.apply(clean)

In [6]:
title_regex = re.compile("== .*? ==\n?")
title_split_regex = re.compile("(== .*? ==\n?)")
invalid_titles = [
    re.compile("== .*?Sources.*? ==\n?"),
    re.compile("== .*?Also.*? ==\n?"),
    re.compile("== .*?Note.*? ==\n?"),
]

def valid_title(title):
    return re.match(title_regex, title) and not any(re.match(r, title) for r in invalid_titles)

def filter_sections(text):
    sections = re.split(title_split_regex, text)
    good_sections = [section for title, section in zip(sections[:-1], sections[1:]) if valid_title(title)]
    return "\n".join(good_sections)

def split_sentences(text):
    return filter_sections(text).split("\n")

split_sentences(clean(df.iloc[0].body))

['Let {inline-equation} be a monoid whose identity is {inline-equation}.',
 'Let {inline-equation}.',
 'Let:',
 '{inline-equation} have a left inverse for {inline-equation}',
 '{inline-equation} have a right inverse for {inline-equation}.',
 'Then both {inline-equation} and {inline-equation} are invertible for {inline-equation}.',
 '',
 'Let {inline-equation} be the left inverse of {inline-equation} and {inline-equation} be the right inverse of {inline-equation}. Then:',
 '{equation}',
 '{equation}',
 'Thus {inline-equation} has both a left inverse {inline-equation} and a right inverse {inline-equation}.',
 'From {theorem}:',
 '{inline-equation}',
 'and {inline-equation} has an inverse, that is, is invertible.',
 'From the above, we have:',
 '{equation}',
 'and:',
 '{equation}',
 'Thus {inline-equation} has both a left inverse {inline-equation} and a right inverse {inline-equation}.',
 'From {theorem}:',
 '{inline-equation}',
 'and {inline-equation} has an inverse, that is, is invertib

In [7]:
sentences = df.body_cleaned.apply(split_sentences).explode()
sentences = sentences[sentences.str.len() > 0]

In [8]:
punctuation = re.compile("(?!\{|-|\})[^\w\s]")

def clean_sentence(sentence):
    sentence = sentence.lower()
    sentence = re.sub(punctuation, "", sentence)
    return sentence

In [9]:
sentences_cleaned = sentences.apply(clean_sentence)

In [10]:
for sentence in sentences_cleaned.iloc[:1000]:
    print("*", sentence)

* let {inline-equation} be a monoid whose identity is {inline-equation}
* let {inline-equation}
* let
* {inline-equation} have a left inverse for {inline-equation}
* {inline-equation} have a right inverse for {inline-equation}
* then both {inline-equation} and {inline-equation} are invertible for {inline-equation}
* let {inline-equation} be the left inverse of {inline-equation} and {inline-equation} be the right inverse of {inline-equation} then
* {equation}
* {equation}
* thus {inline-equation} has both a left inverse {inline-equation} and a right inverse {inline-equation}
* from {theorem}
* {inline-equation}
* and {inline-equation} has an inverse that is is invertible
* from the above we have
* {equation}
* and
* {equation}
* thus {inline-equation} has both a left inverse {inline-equation} and a right inverse {inline-equation}
* from {theorem}
* {inline-equation}
* and {inline-equation} has an inverse that is is invertible
* let {inline-equation} be a preordered set
* let {inline-equ

* {inline-equation} is either equal to {inline-equation} or finer than {inline-equation}
* {inline-equation} is either equal to {inline-equation} or finer than {inline-equation}
* we find
* {inline-equation} by the definition of lower sum and {inline-equation} refining {inline-equation}
* {inline-equation} by {theorem}
* {inline-equation} by the definition of upper sum and {inline-equation} refining {inline-equation}
* by combining these inequalities we conclude
* {inline-equation}
* {inline-equation} two thousand one hundred and seventy-eight is
* {inline-equation}
* the {inline-equation}nd of only {inline-equation} numbers with {inline-equation} digits or fewer which has a multiple which is its reversal
* {inline-equation}
* a {inline-equation}th order recurring digital invariant
* {inline-equation} {inline-equation}
* let {inline-equation} be a field whose unity is {inline-equation}
* let {inline-equation} be a subfield of {inline-equation}
* the unity of {inline-equation} is also {

In [11]:
sentences_cleaned.to_csv("../sentences.csv", header = None, index = False)

# Inspecting for errors

In [12]:
title_regex = re.compile("== .*? ==\n")
title_split_regex = re.compile("(== .*? ==\n)")
invalid_titles = [
    re.compile("== Sources.*? ==\n"),
    re.compile("== Also.*? ==\n"),
    re.compile("== .*?Note.*? ==\n"),
]

def valid_title(title):
    return re.match(title_regex, title) and not any(re.match(r, title) for r in invalid_titles)

def filter_sections(text):
    sections = re.split(title_split_regex, text)
    good_sections = [section for title, section in zip(sections[:-1], sections[1:]) if valid_title(title)]
    return good_sections

def get_pattern(text):
    sections = re.findall(title_split_regex, text)
    #good_sections = [section for title, section in zip(sections[:-1], sections[1:]) if valid_title(title)]
    #return good_sections
    return sections

text = "aa == Proof ==\n bb == 2 ==\n cc == 3 ==\n dd"
filter_sections(text)

[' bb ', ' cc ', ' dd']

In [13]:
tag_counts = df.body_cleaned.apply(get_pattern).explode().value_counts()

In [14]:
for tag, count in tag_counts.iloc[:100].iteritems():
    if valid_title(tag):
        print(tag.strip(), count)

== Proof == 21122
== Theorem == 20718
== Number == 2081
== Proof 1 == 1327
== {theorem} == 969
== Proof 2 == 949
== Necessary Condition == 644
== Sufficient Condition == 635
== Example of {theorem} == 505
== Induction Step == 497
== Corollary == 483
== Basis for the Induction == 476
== Induction Hypothesis == 463
== Corollary to {theorem} == 392
== Definition 1 == 301
== Definition 2 == 288
== Example: {inline-equation} == 281
== Examples == 276
== Lemma == 275
== Examples of Use of {theorem} == 227
== Examples of {theorem} == 211
== Proof 3 == 207
== Example of Use of {theorem} == 203
== Solution == 199
== {inline-equation} implies {inline-equation} == 166
== {inline-equation} == 149
== Definition == 142
== Example of Use of {inline-equation} Function == 137
== Definition 3 == 117
== Formulation 1 == 112
== Definition 1 implies Definition 2 == 103
== General Result == 99
== Definition 2 implies Definition 1 == 97
== Proof of == 96
== Corollary 2 == 89
== Corollary 1 == 87
== Proof Rul

In [15]:
for i, rows in df[df.body_cleaned.str.contains("== .* ==!\n")].iloc[:1].iterrows():
    print(rows.url)
    print(rows.body)
    print("~~~~~~")
    print(rows.body_cleaned)